In [1]:
# pip install pandas-profiling

In [1]:
import pandas as pd
import pandas_profiling
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

C:\Users\dell\anaconda3\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\dell\anaconda3\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\dell\AppData\Local\Temp\ipykernel_16912\3841208057.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st.

In [2]:
bankloans = pd.read_csv("bankloans.csv")
print(bankloans.shape)
print(bankloans.columns)
bankloans.head()

(850, 9)
Index(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt', 'default'],
      dtype='object')


,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0


In [3]:
bankloans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       850 non-null    int64  
 1   ed        850 non-null    int64  
 2   employ    850 non-null    int64  
 3   address   850 non-null    int64  
 4   income    850 non-null    int64  
 5   debtinc   850 non-null    float64
 6   creddebt  850 non-null    float64
 7   othdebt   850 non-null    float64
 8   default   700 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 59.9 KB


In [4]:
bankloans.isnull().sum()

age           0
ed            0
employ        0
address       0
income        0
debtinc       0
creddebt      0
othdebt       0
default     150
dtype: int64

In [5]:
people_with_info = bankloans[bankloans.default.isnull() == False]
people_without_info = bankloans[bankloans.default.isnull() == True]

In [6]:
print("Dims of people_with_info:", people_with_info.shape)
print("Dims of people_without_info:", people_without_info.shape)

Dims of people_with_info: (700, 9)
Dims of people_without_info: (150, 9)


In [7]:
pandas_profiling.ProfileReport(bankloans)

TypeCheckError: argument "config_file" (None) did not match any element in the union:
  pathlib.Path: is not an instance of pathlib.Path
  str: is not an instance of str

In [9]:
def outlier_capping(x):
    x = x.clip(upper=x.quantile(0.99))
    x = x.clip(lower=x.quantile(0.01))
    return x

people_with_info = people_with_info.apply(lambda x: outlier_capping(x))

In [27]:
x_vars = [colname for colname in people_with_info.columns if colname!="default"]
x = people_with_info[x_vars]
y = people_with_info['default'].values

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [29]:
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=5)

In [30]:
rfe_model = rfe.fit(x_train, y_train)

In [31]:
x_train.columns[rfe_model.get_support()]

Index(['employ', 'income', 'debtinc', 'creddebt', 'othdebt'], dtype='object')

In [46]:
x_train.columns

Index(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt'],
      dtype='object')

In [32]:
from sklearn.model_selection import GridSearchCV

In [36]:
params = {
    "n_estimators": [10, 20, 40, 80, 100],
    "max_depth": [1, 2, 3]
}

rf_estimator = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=1024)

In [37]:
cv_rf = GridSearchCV(rf_estimator, params, cv=5)
cv_rf.fit(x_train, y_train)

C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do no

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=1024),
             param_grid={'max_depth': [1, 2, 3],
                         'n_estimators': [10, 20, 40, 80, 100]})

In [38]:
cv_rf.best_estimator_

RandomForestClassifier(max_depth=3, n_estimators=10, n_jobs=-1, oob_score=True,
                       random_state=1024)

In [39]:
from sklearn.metrics import classification_report

In [43]:
print(classification_report(y_train, cv_rf.best_estimator_.predict(x_train)))

              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88       344
         1.0       0.76      0.38      0.50       125

    accuracy                           0.80       469
   macro avg       0.78      0.67      0.69       469
weighted avg       0.79      0.80      0.78       469



In [45]:
print(classification_report(y_test, cv_rf.best_estimator_.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.79      0.95      0.86       173
         1.0       0.62      0.22      0.33        58

    accuracy                           0.77       231
   macro avg       0.70      0.59      0.60       231
weighted avg       0.74      0.77      0.73       231



In [64]:
x_vars = [colname for colname in people_with_info.columns if colname in ['employ', 'income', 'debtinc', 'creddebt', 'othdebt']]
x = people_with_info[x_vars]
y = people_with_info['default'].values

In [65]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [66]:
params = {
    "n_estimators": [10, 20, 40, 80, 100],
    "max_depth": [1, 2, 3]
}

rf_estimator = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=1024)

In [67]:
cv_rf = GridSearchCV(rf_estimator, params, cv=5)
cv_rf.fit(x_train, y_train)

C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\amand\.conda\envs\dev\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do no

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=1024),
             param_grid={'max_depth': [1, 2, 3],
                         'n_estimators': [10, 20, 40, 80, 100]})

In [68]:
cv_rf.best_estimator_

RandomForestClassifier(max_depth=2, n_estimators=10, n_jobs=-1, oob_score=True,
                       random_state=1024)

In [69]:
print(classification_report(y_train, cv_rf.best_estimator_.predict(x_train)))

              precision    recall  f1-score   support

         0.0       0.80      0.95      0.87       344
         1.0       0.74      0.36      0.48       125

    accuracy                           0.80       469
   macro avg       0.77      0.66      0.68       469
weighted avg       0.79      0.80      0.77       469



In [70]:
print(classification_report(y_test, cv_rf.best_estimator_.predict(x_test)))

              precision    recall  f1-score   support

         0.0       0.81      0.95      0.88       173
         1.0       0.71      0.34      0.47        58

    accuracy                           0.80       231
   macro avg       0.76      0.65      0.67       231
weighted avg       0.79      0.80      0.77       231



In [71]:
import pickle

In [72]:
filename = "RF_model_example.pkl"
pickle.dump(cv_rf.best_estimator_, open(filename, 'wb'))

In [73]:
retrived_model = pickle.load(open(filename, 'rb'))

In [74]:
retrived_model.predict(x_train)

array([0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [75]:
x_train.columns

Index(['employ', 'income', 'debtinc', 'creddebt', 'othdebt'], dtype='object')